In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2

In [2]:
start = time.time()
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [3]:
def split_data(x, y, ratio,seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    # ***************************************************
    indices = np.random.permutation(x.shape[0])
    training_ratio = int(np.floor(ratio * x.shape[0]))

    x_training = x[indices[0:training_ratio]]
    y_training = y[indices[0:training_ratio]]
    x_testing = x[indices[training_ratio:]]
    y_testing = y[indices[training_ratio:]]
    
    return x_training, x_testing, y_training, y_testing

In [4]:
# Removing bothering data and centering
tX[tX==-999] = 0
m = np.mean(tX, axis=0)
centered_tX = tX - m

centered_tX[centered_tX==0] = float('nan')
stdevtrain = np.nanstd(centered_tX, axis=0)
centered_tX[centered_tX==float('nan')] = 0
standardized_tX = centered_tX / stdevtrain

d = len(standardized_tX[0])
n = len(standardized_tX)
indices_s_deg = []

# Creating indices for subsets of degree 2
for i in range (d):
    for t in range (i,d): 
        indices_s_deg.append([t, i])
indices_s_deg = np.array(indices_s_deg).T

degrees = range(3,11)
degrees_number = len(degrees) + 1
stdX_Ncols = standardized_tX.shape[1]
indices_s_Ncols = indices_s_deg.shape[1]
number_of_rows = indices_s_Ncols + degrees_number * stdX_Ncols

mat = np.zeros((n, number_of_rows))

# First degree
mat[:, :stdX_Ncols] = standardized_tX

# Second degree gotten from indices
mat[:,stdX_Ncols:stdX_Ncols + indices_s_Ncols] = standardized_tX[:, indices_s_deg[0]] * standardized_tX[:, indices_s_deg[1]]

# Improve 3 to 10 degree
for i in degrees:
    start_index = indices_s_Ncols + (i - 2) * stdX_Ncols
    end_index = start_index + stdX_Ncols
    mat[:,start_index:end_index] = standardized_tX**i

In [5]:
#excluded=[]
#for i in range (len(mat[0])):
#    a=np.corrcoef(y,mat[:,i])
#    if abs(a[0,1])<10**-3:
#        excluded.append(i)

In [6]:
#mat=np.delete(mat,excluded,axis=1)

In [7]:
m2 = np.mean(mat, axis=0)
centered_mat = mat - m2
centered_mat[mat==0] = 0

centered_mat[centered_mat==0] = float('nan')
stdev = np.nanstd(centered_mat, axis=0)
centered_mat[centered_mat==float('nan')] = 0
standardized_mat = centered_mat / stdev

In [8]:
num_samples = len(standardized_mat)
tx = np.c_[np.ones(num_samples), standardized_mat]
#y=list(y)
#y[y==-1]=0

In [9]:
def compute_loss(y, tx, w):
    """Calculate the loss.
    
    You can calculate the loss using mse or mae.
    """
    e = y - np.dot(tx, w)
    mse = np.dot(e.transpose(), e) / (2 * len(tx))
    return mse

In [10]:
tX, testx, y, testy = split_data(tx, y, 0.8, seed=1)

In [11]:
from __future__ import division
import numpy as np 
def logistic(a):
    return 1.0 / (1 + np.exp(-a))
def irls(X, y):
    theta = np.zeros(X.shape[1])
    theta_ = np.inf
    eps = 60000
    lamda = 5
    for aqua in range (11):
        grad = np.zeros(X.shape[1])
        a = np.dot(X, theta)
        #a=math.log10(abs(a))*sign(a)
        #print(min(a),max(a))
        pi = logistic(a)        
        SX = X * (pi - pi*pi).reshape(-1, 1)
        XSX = np.dot(X.T, SX)
        #+lamda*np.eye((len(X[0])))
        for aw in range (len(X)):
            grad = grad + (-1 / len(X)) * (y[aw] * X[aw, :] * logistic(-y[aw] * np.dot(X[aw,:], theta)))
        
        
     
        theta = theta - eps * np.linalg.solve(XSX, grad)
        print(sum(y == np.sign(np.dot(X, theta))) / len(y))
       
        if aqua % 5 == 0 and aqua != 0:
            eps = eps * 0.8
    return theta

In [12]:
theta=irls(tX,y)

0.82505
0.826655
0.828295
0.829095
0.831
0.831925
0.832935
0.833575


/home/romain/.local/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in exp


0.834155
0.83477
0.834845


In [13]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [14]:
print("Freeing memory")
del(mat)
del(DATA_TRAIN_PATH)
del(y)
del(tx)
del(ids)
weights = theta
testx = tX_test
#testx=np.delete(testx,[14,15,17,18,24,25,27,28],axis=1)
testx[testx==-999] = 0
#m2=np.mean(testx,axis=0)
centered_testx = testx - m
centered_testx[testx==-999] = 0
#standardized_testx=centered_testx / np.std(centered_testx, axis=0)
#centered_testx[centered_testx==0]=float('nan')
#stdevtest=np.nanstd(centered_testx,axis=0);
#centered_testx[centered_testx==float('nan')]=0
standardized_testx = centered_testx / stdevtrain

d = len(standardized_testx[0])
n = len(standardized_testx)
indices_s_deg = []

# Creating indices for subsets of degree 2
for i in range (d):
    for t in range (i,d): 
        indices_s_deg.append([t, i])
indices_s_deg = np.array(indices_s_deg).T

degrees = range(3,11)
degrees_number = len(degrees) + 1
stdX_Ncols = standardized_testx.shape[1]
indices_s_Ncols = indices_s_deg.shape[1]
number_of_rows = indices_s_Ncols + degrees_number * stdX_Ncols

mat = np.zeros((n, number_of_rows))

# First degree
mat[:, :stdX_Ncols] = standardized_testx

# Second degree gotten from indices
mat[:,stdX_Ncols:stdX_Ncols + indices_s_Ncols] = standardized_testx[:, indices_s_deg[0]] * standardized_testx[:, indices_s_deg[1]]

# Improve 3 to 10 degree
for i in degrees:
    start_index = indices_s_Ncols + (i - 2) * stdX_Ncols
    end_index = start_index + stdX_Ncols
    mat[:,start_index:end_index] = standardized_testx**i  
        
centered_mat = mat - m2
centered_mat[mat==0] = 0

print("Freeing the matrix memory again")
del(mat)

#centered_mat[centered_mat==0]=float('nan')
#stdev=np.nanstd(centered_mat,axis=0);
#centered_mat[centered_mat==float('nan')]=0
standardized_testmat = centered_mat / stdev

#tao=int(d*(d+1)/2+d)
    
#for i in range (n):
#    for r in range (d):
#        for monsoon in range (r,d):
#            for t in range (monsoon,d): 
#                mat[i,tao]=standardized_tX[i,t]*standardized_tX[i,monsoon]*standardized_tX[i,r]
#                tao=tao+1
#    tao=int(d*(d+1)/2+d)  
    
num_samples = len(standardized_testmat)
final_testx = np.c_[np.ones(num_samples), standardized_testmat]
tX_test = final_testx

Freeing memory
Freeing the matrix memory again


In [15]:
OUTPUT_PATH = 'deneme8.csv' # TODO: fill in desired name of output file for submission
y_pred = -predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [16]:
end = time.time()
print("New time for everything:", end - start, "seconds.")

New time for everything: 180.27915120124817 seconds.
